## Icolos Docking Workflow Demo
In this notebook, we demonstrate an Icolos workflow for automated molecular dynamics simulations of a protein-ligand complex using GROMACS.  The only required files are the `mdp` files containing the run settings for each stage, and a prepared pdb file suitable for gromacs.  Ligands and cofactors are automatically parametrised with GAFF through the `acpype` wrapper for Antechamber.

Files required to execute the workflow are provided in the accompanying IcolosData repository, available at https://github.com/MolecularAI/IcolosData.

Note, we provide an `icoloscommunity` environment which should be used for this notebook.  It contains the jupyter dependencies in addition to the Icolos production environment requirements, allowing you to execute workflows from within the notebook

### Step 1: Prepare the run
The following files are required to start the MD run
* A prepared pdb file 
* mdp file corresponding to each simulation component
* Icolos config file: a `JSON` file containing the run settings.  Templates for the most common workflows can be found in the `examples` folder of main Icolos repository.

This workflow requires you to have the following packages installed:
* GROMACS

For clusters using a module system, use prefix_execution to load the required module into your $PATH for that step.

Note we set a few environment variables to offload virtually all of the computation to the GPU where available

In [1]:
import os
import json
import subprocess
import pandas as pd

# set up some file paths to use the provided test data
# please ammend as appropriate
icolos_path = "~/Icolos"
data_dir = "~/IcolosData"
output_dir = os.path.abspath("../output")

config_dir = "../config/gromacs"
for path in [output_dir, config_dir]:
    os.makedirs(path, exist_ok=True)
pdb_path = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/1BVG.pdb"))
ions_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/ions.mdp"))
em_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/minim.mdp"))
nvt_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/nvt_equil.mdp"))
npt_mdp = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/npt_equil.mdp"))
prod_md = os.path.expanduser(os.path.join(data_dir, "gromacs/protein/md.mdp"))




In [2]:
conf={
    "workflow": {
        "header": {
            "workflow_id": "gromacs_test",
            "description": "full md run with gromacs",
            "environment": {
                "export": [{
                    "key": "GMX_GPU_DD_COMMS",
                    "value": "true"
                }, {
                    "key": "GMX_GPU_PME_PP_COMMS",
                    "value": "true"
                }, {
                    "key": "GMX_FORCE_UPDATE_DEFAULT_GPU",
                    "value": "true"
                }
                ]
            },
            "global_variables": {
                
            }
        },
        "steps": [{
            "step_id": "01_pdb2gmx",
            "type": "pdb2gmx",
            "execution": {
                "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"

            },
            "settings": {
                "arguments": {
                    "flags": ["-ignh"],
                    "parameters": {
                        "-water": "tip3p",
                        "-ff": "amber03"
                    }
                },
                "additional": {
                    "charge_method": "gas"
                }
            },
            "input": {
                "generic": [
                    {
                        "source": pdb_path,
                        "extension": "pdb"
                    }
                ]}
        },{
            "step_id": "02_editconf",
            "type": "editconf",
            "execution": {
                "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
            },
            "settings": {
                "arguments": {
                    "flags": ["-c"],
                    "parameters": {
                        "-d": "1.0",
                        "-bt": "dodecahedron"
                    }
                }
            }
        },{
            "step_id": "03_solvate",
            "type": "solvate",
            "execution": {
                "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
            },
            "settings": {
                "arguments": {
                    "flags": [],
                    "parameters": {
                        "-cs": "spc216"
                    }
                }
                }
        },
            {
                "step_id": "04_grompp",
                "type": "grompp",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                        }
                    },
                    "additional": {
                        "-r": False
                    }
                },
                "input": {
                    "generic": [
                        {
                            "source": ions_mdp,
                            "extension": "mdp"
                        }
                    ]}

            },{
                "step_id": "05_genion",
                "type": "genion",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": ["-neutral"],
                        "parameters": {
                            "-pname": "NA",
                            "-nname": "CL"
                        }
                    },
                    "additional": {
                        "pipe_input": "SOL"
                    }
                }

            },{
                "step_id": "06_grompp_eminim",
                "type": "grompp",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {

                        }
                    },
                    "additional": {
                        "-r": False
                    }
                },
                "input": {
                    "generic": [
                        {
                            "source": em_mdp,
                            "extension": "mdp"
                        }
                    ]}
            },{
                "step_id": "07_eminim_mdrun",
                "type": "mdrun",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {

                        }
                    },
                    "additional": {
                    }
                }
            },{
                "step_id": "08_nvt_grompp",
                "type": "grompp",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                            "-n": "index.ndx"
                        }
                    },
                    "additional": {
                        "make_ndx_command": "auto",
                        "-r": True
                    }
                },
                "input": {
                    "generic": [
                        {
                            "source": nvt_mdp,
                            "extension": "mdp"
                        }
                    ]}
            },{
                "step_id": "09_nvt_mdrun",
                "type": "mdrun",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                        }
                    },
                    "additional": {
                    }
                }
            },{
                "step_id": "10_npt_grompp",
                "type": "grompp",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                            "-n": "index.ndx"
                        }
                    },
                    "additional": {
                        "make_ndx_command": "auto",
                        "-r": True
                    }
                },
                "input": {
                    "generic": [
                        {
                            "source": npt_mdp,
                            "extension": "mdp"
                        }
                    ]}
            },{
                "step_id": "11_npt_mdrun",
                "type": "mdrun",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                        }
                    },
                    "additional": {
                        "make_ndx_command": "auto"
                    }
                }

            },{
                "step_id": "12_prod_md_grompp",
                "type": "grompp",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                            "-n": "index.ndx"
                        }
                    },
                    "additional": {
                        "fields": {
                            "nsteps": "50000"
                        },
                        "make_ndx_command": "auto",
                        "-r": False
                    }
                },
                "input": {
                    "generic": [
                        {
                            "source": prod_md,
                            "extension": "mdp"
                        }
                    ]
                }
            },{
                "step_id": "13_prod_mdrun",
                "type": "mdrun",
                "execution": {
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings": {
                    "arguments": {
                        "flags": [],
                        "parameters": {
                            "-nb": "gpu",
                            "-bonded": "gpu"
                        }
                }
                },
                "writeout": [
                    {
                        "generic": {
                            "key": "xtc"
                        },
                        "destination": {
                            "resource": os.path.join(output_dir, "md.xtc"),
                            "type": "file",
                            "format": "TXT"
                        }
                    },
                    {
                        "generic": {
                            "key": "log"
                        },
                        "destination": {
                            "resource": os.path.join(output_dir, "md.log"),
                            "type": "file",
                            "format": "TXT"
                        }
                    },
                    {
                        "generic": {
                            "key": "gro"
                        },
                        "destination": {
                            "resource": os.path.join(output_dir, "md.gro"),
                            "type": "file",
                            "format": "TXT"
                        }
                    },
                    {
                        "generic": {
                            "key": "tpr"
                        },
                        "destination": {
                            "resource": os.path.join(output_dir, "md.tpr"),
                            "type": "file",
                            "format": "TXT"
                        }
                    }
                ]
            },{
                "step_id": "14_trjconv",
                "type": "trjconv",
                "execution":{
                    "prefix_execution": "module load GROMACS/2021-fosscuda-2019a-PLUMED-2.7.1-Python-3.7.2"
                },
                "settings":{
                    "arguments":{
                        "flags":["-center"]
                    },
                    "additional":{
                        "pipe_input": "Protein System"
                    }
                },
                
                "writeout":[
                    {
                        "generic": {
                            "key": "xtc"
                        },
                        "destination": {
                            "resource": os.path.join(output_dir, "trjconv.xtc"),
                            "type": "file",
                            "format": "TXT"
                        }
                    }
                ]
            } 
        ]
    }
}


with open(os.path.join(config_dir, "gromacs_md_conf.json"), 'w') as f:
    json.dump(conf, f, indent=4)

The workflow can be executed by running the following command (with paths ammended as necessary), in a terminal. 

In [3]:
# this run will take a few minutes to complete with a GPU.  You can watch icolos_run.log to check it's progress
conf = os.path.join(config_dir, "gromacs_md_conf.json")

command = f"icolos -conf {conf} -debug"
subprocess.run(command, shell=True)

WARNING - Could not import module espsim, check it is installed in your environment
                                                                                                                                                 
                                        ooooo   .oooooo.     .oooooo.   ooooo          .oooooo.    .oooooo..o                                         
                                        `888'  d8P'  `Y8b   d8P'  `Y8b  `888'         d8P'  `Y8b  d8P'    `Y8                                         
                                         888  888          888      888  888         888      888 Y88bo.                                              
                                         888  888          888      888  888         888      888  `"Y8888o.                                          
                                         888  888          888      888  888         888      888      `"Y88b                                         
               

CompletedProcess(args='icolos -conf ../config/gromacs/gromacs_md_conf.json -debug', returncode=0)

You can now view the resulting trajectory files using VMD or similar